# Milestone 3 - Data Cleaning & Connect to Elasticsearch
---
Name: Rhesa Akbar Elvarettano  

Batch: SBY-003

Introduction:
Tujuan utama dari notebook ini adalah untuk melakukan cleaning data untuk & diconnectkan pada elasticsearch agar dapat membuay visualisasi pada kibana.

Dataset: https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset


# 1 - Import Libraries

In [1]:
import datetime as dt
from datetime import timedelta
from datetime import datetime

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator

import pandas as pd
import psycopg2 as db
from elasticsearch import Elasticsearch

# Connect to Postgres
from sqlalchemy import create_engine

C:\Users\User\AppData\Local\Temp\ipykernel_9412\40907178.py:6 DeprecationWarning: This module is deprecated. Please use `airflow.operators.bash`.

C:\Users\User\AppData\Local\Temp\ipykernel_9412\40907178.py:7 DeprecationWarning: This module is deprecated. Please use `airflow.operators.python`.

# 2 - Data Loading

In [ ]:
# conn_string="dbname='postgres' host='localhost' user='postgres' password='123456"
# conn=db.connect(conn_string)
# df=pd.read_sql("select * from table_m3",conn)

# df

In [11]:
# load data asli
df_ori = pd.read_csv('P2M3_rhesa_akbar_data_raw.csv')

In [12]:
# Untuk menjaga  data asli supaya tidak berubah dan hilang
df = df_ori.copy()

In [13]:
# Untuk menampilkan semua kolom
pd.set_option('display.max_columns', None)

# Menampilkan dataset credit card
df

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly


In [14]:
# Menampilkan 5 baris teratas dari dataset
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [15]:
# Menampilkan 5 baris terbawah dari dataset
df.tail()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly
3899,3900,52,Female,Handbag,Accessories,81,California,M,Beige,Spring,3.1,No,Store Pickup,No,No,33,Venmo,Quarterly


# 3 - Data Cleaning

In [16]:
# Menampilkan informasi tentang tipe data, missing value dan info lainnya.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Shipping Type             3900 non-null   object 
 13  Discount Applied          3900 non-null   object 
 14  Promo Co

Berdasarkan informasi diatas bahwasanya tidak terdapat missing value pada dataset.

In [17]:
# check duplicated data
df.duplicated().sum()

0

Setelah dilakukan pengecekan pada jumlah data duplikat, hasilnya telah 0 artinya sudah tidak ada data duplikat

In [19]:
columns = df.columns # Mengambil daftar nama kolom dari dataframe df
new_col_names = {} # Membuat kamus kosong yang akan digunakan untuk memetakan nama kolom lama ke nama kolom baru setelah pembersihan

for col in columns:
        col_remove_char = col.replace('(', '').replace(')', '') # Menghapus tanda kurung dari nama kolom jika ada

        col_split = col_remove_char.split(' ')
        for i in range(len(col_split)):
            word_lower = col_split[i].lower() # Setiap kata diubah menjadi huruf kecil
            col_split[i] = word_lower
        new_col_name = '_'.join(col_split) # Setelah semua kata diubah menjadi huruf kecil, kata-kata tersebut digabungkan kembali menjadi satu string dengan menggunakan _ sebagai pemisah

        new_col_names[col] = new_col_name

df = df.rename(columns = new_col_names)

In [20]:
# Setelah dilakukan missing value, duplikat data, dan cleaning data, dicek kembali nama-nama kolomnya
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_usd', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'preferred_payment_method', 'frequency_of_purchases'],
      dtype='object')

In [24]:
# Saving to new csv
df.to_csv('P2M3_rhesa_akbar_data_clean.csv', index=False) # Menyimpan dataframe df yang telah dimodifikasi ke dalam file CSV baru 

# 4 - Elasticsearch

In [26]:
# Load data to elasticsearch
data = pd.read_csv('P2M3_rhesa_akbar_data_clean.csv')

In [27]:
data.head()

,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,preferred_payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [30]:
# Establish to connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")
es.ping()

[2024-03-23 14:55:02,115] {base.py:270} INFO - HEAD http://localhost:9200/ [status:200 request:0.964s]


True

In [31]:
for i,r in data.iterrows():
    doc=r.to_json()
    res=es.index(index="manual_elasticsearch",doc_type="doc",body=doc)
    print(res)    

c:\Users\User\miniconda3\envs\docker\lib\site-packages\elasticsearch\connection\base.py:208 ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).

[2024-03-23 14:55:13,206] {base.py:270} INFO - POST http://localhost:9200/manual_elasticsearch/doc [status:201 request:2.874s]
{'_index': 'manual_elasticsearch', '_type': 'doc', '_id': 'siNOao4BJf7xD7T-fXIe', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
[2024-03-23 14:55:13,241] {base.py:270} INFO - POST http://localhost:9200/manual_elasticsearch/doc [status:201 request:0.024s]
{'_index': 'manual_elasticsearch', '_type': 'doc', '_id': 'syNOao4BJf7xD7T-f3LG', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
[2024-03-23 14:55:13,318] {base.py:270} INFO - POST http://localhost:9200/manual_elasticsearch/doc [status:201 request:0.074s]
{'_index': 'manual_elasticsearch', '_type': 'doc', '_id': 'tCNOao4BJf7xD7T-gHIR', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1